<a href="https://colab.research.google.com/github/LordNJS/fai-practical/blob/main/Best_first_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Best First Search

In [19]:
import copy
import heapq

def print_sudoku(sudoku):
  #format and print
  count_row = 0
  count_col = 0
  for a in sudoku:
    if count_row == 3 or count_row == 6:
      print("------+-------+------")
    for b in a:
      print(b, end= " ")
      if count_col == 2 or count_col == 5:
        print("|", end=" ")
      count_col += 1
    count_col = 0
    count_row += 1
    print()

class sudoku:
  # define possible states

  frontier = []
  explored_set = []
  possible_nums = [1,2,3,4,5,6,7,8,9]

  #heuristic counts how many rules are broken
  # the sudoku board fixed numbers are doubled, eg. 11,22,33,44
  # this is to easily find which are fixed and which can be changed after filling the board
  def heuristic(self,sudoku,x,y):

    if x == y and x == 999:
      the_num = 0
    else:
      the_num = sudoku[y][x]
    the_square_num = y // 3 * 3 + x // 3
    rule_broken = 0
    row_num = 0
    for i in sudoku:
      # this is to not change sudoku when trying to get the fixed value
      rowl = [];
      for a in i:
        rowl.append(a%10)
      row = set(rowl)
      for a in row:
        if a != 0:
          rule_broken += rowl.count(a) - 1
        if row_num == y:
          rule_broken += (rowl.count(the_num) -1) * 5
      row_num += 1

    #get columns
    sudoku_cols = []
    for i in range(9):
      col = []
      for a in range(9):
        col.append(sudoku[a][i]%10)
      sudoku_cols.append(col)

    col_num = 0
    for i in sudoku_cols:
      col = set(i)
      for a in col:
        if a != 0:
            rule_broken += i.count(a) - 1
      if col_num == x:
        rule_broken += (i.count(the_num) - 1) * 5
      col_num += 1

    #get squares
    sudoku_squares = []
    squares = [[],[],[]]
    for i in range(9):
      for j in range(9):
        squares[j//3].append(sudoku[i][j]%10)
      if i % 3 == 2:
        sudoku_squares += squares
        squares = [[],[],[]]

    square_num = 0
    for i in sudoku_squares:
      square = set(i)
      for a in square:
        if  a != 0:
          rule_broken += i.count(a) - 1
      if square_num == the_square_num:
        rule_broken += (i.count(the_num) - 1) * 5
      square_num += 1

    return rule_broken

    # get numbers that are not fixed (eg. fixed = 1,2; remaining_num = 3,4,5,6,7,8,9)
  def remaining_num(self,row):
    num_fixed = []
    #get fixed
    for i in row:
      if i > 10:
        num_fixed.append(i%10)

    nums = [1,2,3,4,5,6,7,8,9]
    nums = [n for n in nums if n not in num_fixed]

    return nums

  #fill the board
  def fill(self,sudoku):
    for i in sudoku:
      for j in range(9):
        if i[j] != 0:
          i[j]=i[j] * 11

    for i in sudoku:
      add_num = self.remaining_num(i)
      for j in add_num:
        for x in range(len(i)):
          #fill if empty
          if i[x] == 0:
            i[x] = j
            break

    return sudoku

  def searching(self, sudoku, x, y):
    if sudoku[y][x] > 10:
      return

    new_sudoku = copy.deepcopy(sudoku)

    for a in self.possible_nums:
      new_sudoku[y][x] = a
      if new_sudoku in self.explored_set:
        continue
      rule_broken = self.heuristic(new_sudoku,x,y)
      if (rule_broken,new_sudoku) in self.frontier:
        continue
      heapq.heappush(self.frontier, (rule_broken, copy.deepcopy(new_sudoku)))

  def __init__(self, sudoku):
    if self.heuristic(sudoku,999,999) != 0:
      print("A rule is already broken")
      return

    sudoku = self.fill(sudoku)

    x = 0
    y = 0

    current_rule_broken = self.heuristic(sudoku,0,0)
    heapq.heappush(self.frontier, (current_rule_broken, sudoku))
    iteration = 0
    while current_rule_broken != 0:
      self.searching(sudoku,x,y)
      a = heapq.heappop(self.frontier)

      print_sudoku(a[1])
      print("heuristic: ", a[0])
      print("iteration: ", iteration)
      print()
      iteration += 1

      sudoku = a[1]
      self.explored_set.append(copy.deepcopy(a[1]))
      current_rule_broken = a[0]
      y += 1
      if y == 9:
        y = 0
        x += 1
        if x == 9:
          x = 0

In [20]:
asudoku = [
[9,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,5,0,0,0,0],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,1],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0],
[0,0,0,0,0,0,0,0,0]]

sudoku(asudoku);

Streaming output truncated to the last 5000 lines.
iteration:  3925

99 7 2 | 3 4 1 | 5 6 8 
3 4 6 | 1 7 5 | 2 7 9 
5 7 1 | 2 8 4 | 6 3 9 
------+-------+------
7 9 5 | 8 55 9 | 7 6 3 
1 2 3 | 4 5 6 | 7 8 9 
2 3 4 | 5 6 7 | 8 9 11 
------+-------+------
6 9 5 | 9 1 2 | 4 7 6 
1 2 3 | 4 5 6 | 7 8 9 
4 5 7 | 8 1 3 | 1 7 5 
heuristic:  77
iteration:  3926

99 7 2 | 3 4 1 | 5 6 8 
3 4 6 | 1 7 5 | 2 7 9 
5 7 1 | 2 8 4 | 6 3 9 
------+-------+------
8 9 5 | 8 55 9 | 7 6 3 
1 2 3 | 4 5 6 | 7 8 9 
2 3 4 | 5 6 7 | 8 9 11 
------+-------+------
6 9 5 | 9 1 2 | 4 7 6 
1 2 3 | 4 5 6 | 7 8 9 
4 5 8 | 8 1 3 | 1 3 5 
heuristic:  77
iteration:  3927

99 7 2 | 3 4 1 | 5 6 8 
3 4 6 | 1 7 5 | 2 7 9 
5 7 1 | 2 8 4 | 6 3 9 
------+-------+------
8 9 5 | 8 55 9 | 7 6 3 
1 2 3 | 4 5 6 | 7 8 9 
2 3 4 | 5 6 7 | 8 9 11 
------+-------+------
6 9 5 | 9 1 2 | 4 7 6 
1 2 3 | 4 5 6 | 7 8 9 
4 5 8 | 8 1 3 | 1 4 5 
heuristic:  77
iteration:  3928

99 7 2 | 3 4 1 | 5 6 8 
3 4 6 | 8 7 5 | 2 7 9 
5 7 1 | 2 8 4 | 6 3 9 


KeyboardInterrupt: 